In [ ]:
import numpy as np
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Convolution2D, MaxPooling2D  
from keras.optimizers import SGD
from keras.utils import np_utils
from scipy import misc
import glob
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.color import rgb2gray

In [ ]:
def get_pixel_values():
#     file_name = r''
    pixels = []
    cnt = 0
    for filename in glob.glob('new_images/'+'\*.png'):
        im = misc.imread(filename)
        pixels.append(im)
        cnt+=1
    print(cnt)
    return pixels
def convert_image():
#     file_name = r'\figures_v2'
    for filename in glob.glob('images_2/' + '\*.png'):
        img = Image.open(filename)
#         img = rgb2gray(img);
        img = img.convert('RGB')
        img.save(filename)
# convert_image()
pixel_values = np.asarray(get_pixel_values())

In [ ]:
pixel_values.shape

In [ ]:
import requests
import pandas as pd
import arrow
import datetime
from datetime import timezone
import os
import quandl
import numpy as np
from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def get_quote_data(symbol='AAPL', data_range='2y', data_interval='1h'):
    res = requests.get('https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?range={data_range}&interval={data_interval}'.format(**locals()))
    data = res.json()
    body = data['chart']['result'][0]    
    dt = datetime.datetime

    dt = pd.Series(map(lambda x: arrow.get(x).datetime.replace(tzinfo=None), body['timestamp']), name='Datetime')
    df = pd.DataFrame(body['indicators']['quote'][0], index=dt)
    dg = pd.DataFrame(body['timestamp'])    
    df = df.loc[:, ('open', 'high', 'low', 'close', 'volume')]
    df.dropna(inplace=True)     #removing NaN rows
    df.columns = ['OPEN', 'HIGH','LOW','CLOSE','VOLUME']    #Renaming columns in pandas

    return df

In [ ]:
final_df = get_quote_data()
final_df['Ticker'] = 'AAPL'
list_of_tickers = ['GOOGL','MSFT','AMZN','FB','BABA','JNJ','JPM','BAC','WMT']
for ticker in list_of_tickers:
    stock_df = get_quote_data(symbol=ticker)
    stock_df['Ticker'] = ticker
    final_df = pd.concat([final_df,stock_df])
final_df.head()

In [ ]:
count = 0
y_10_days_prediction = []
pixel_index = 0

try:
    for index, row in final_df.iterrows():
        if((count-1)%7==0):
            row1 = final_df.iloc[[count+10]]
            y_10_days_prediction.append(np.array(row1["OPEN"])[0])
        count+=1
except:
    pass
    

In [ ]:
len(y_10_days_prediction)

In [ ]:
pixel_values = pixel_values[0:len(y_10_days_prediction)]
print(len(pixel_values))
print(len(y_10_days_prediction))

In [ ]:
x_train = pixel_values[0:4000]
y_train = y_10_days_prediction[0:4000]
x_test = pixel_values[4000:]
y_test = y_10_days_prediction[4000:]
#    y_true = y_test
#    y_train = np_utils.to_categorical(y_train, 2)
#    y_test = np_utils.to_categorical(y_test, 2)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

In [ ]:
def create_model():
    model = Sequential()

    model.add(Convolution2D(32, 3, 3,
                            border_mode='valid', 
                            input_shape=(360, 360, 4)))  
    model.add(Activation('relu'))  
    model.add(Convolution2D(32, 3, 3))  
    model.add(Activation('relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    model.add(Dropout(0.25))  
      
    model.add(Convolution2D(64, 3, 3, 
                            border_mode='valid'))  
    model.add(Activation('relu'))  
    model.add(Convolution2D(64, 3, 3))  
    model.add(Activation('relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    model.add(Dropout(0.25))  
      
    model.add(Flatten())  
    model.add(Dense(256))  
    model.add(Activation('relu'))  
    model.add(Dropout(0.5))

    model.add(Dense(2))  
    model.add(Activation('softmax'))  

    return model
def compile_model(model):
    lrate = 0.01
    sgd = SGD(lr=lrate, momentum=0.9, decay=1e-6, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd)
    return model

In [ ]:
model = create_model()
model = compile_model(model)

In [ ]:
def r_squared(y_true, y_hat):
    ssr = 0
    sst = 0
    e = np.subtract(y_true, y_hat)
    y_mean = np.mean(y_true)
    for item in e:
        ssr += item**2
    for item in y_true:
        sst += (item - y_mean)**2
    r2 = 1 - ssr / sst
    return r2

In [ ]:
# Fit the model
epochs = 10
model.fit(x_train, y_train, validation_data=(x_test, y_test), 
          nb_epoch=epochs,
          shuffle=True, batch_size=100, verbose=1)

In [ ]:
#    scores = model.evaluate(x_test, y_test, verbose=0)
#    print('Accuracy: %.2f%%' % (scores[1] * 100))
classes = model.predict_classes(x_test, verbose=0)

In [ ]:
classes = list(classes)
y_test = list(y_test)
r2 = r_squared(y_test, classes)
print(r2)